<h1 style="font-family:Arial;color:#333399">Python API call to Retrieve Referece Data from A7 API</h1>

<p style="color:#333333;font-family:Arial">
This article is designed to show users how to use A7 API to download data. We cover simple calls to extract reference data needed for specfic contracts.   
</p>
<p style="color:#333333;font-family:Arial">
The following segment of code is to setup the API parameters.    
</p>

In [1]:
proxies = {
"http":  "", # Enter http Proxy if needed
"https": "", # Enter https Proxy if needed
}
 
API_TOKEN="Bearer "+"Enter API Token" 

In [41]:
import pandas as pd
import requests
import numpy as np
from pprint import pprint

<p style="color:#333333;font-family:Arial">
First step would be extract which markets are available. A7 covers Eurex (XEUR) and Xetra (XETR).   
</p>

In [42]:
url = 'https://a7.deutsche-boerse.com/api/v1/rdi'
r = requests.get(url = url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
pprint(r.json())

{'MarketIDs': ['XETR', 'XEUR']}


<p style="color:#333333;font-family:Arial">
Once the available markets have been established, we can extract market segment ID's for contracts of interest. For example which market segment ID contains FGBL, FGBM or FDAX. 
</p>
<p style="color:#333333;font-family:Arial">
Therefore the first step is to pull from the RDI API a data struture (JSON format) that contains market segments. This call requires a date and a level of data description (e.g. reference - Receive reference ids (default), keys - Receive functional keys (like strike price, put/call), detailed - Receive all stored information):        
</p>

In [57]:
reference_date= '20200106' #Note :: Reference data has the formate of yyyymmdd
url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XEUR/{}?mode=detailed'.format(reference_date)
r = requests.get(url = url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
res = r.json()

<p style="color:#333333;font-family:Arial">
Now we can search the data struture for the relavant contract. Please note there are various ways to search the extracted data struture, the following is example of one method.    
</p>

In [58]:
lst_ms = np.array([x['MarketSegment'] for x in res['MarketSegments']])
indx_fgbl = np.where(lst_ms=='FGBL')[0][0]
indx_fdax = np.where(lst_ms=='FDAX')[0][0]
print('Market Segment for FGBL :: '+str(res['MarketSegments'][indx_fgbl]['MarketSegmentID']))
print('Market Segment for FDAX :: '+str(res['MarketSegments'][indx_fdax]['MarketSegmentID']))

Market Segment for FGBL :: 688
Market Segment for FDAX :: 589


<p style="color:#333333;font-family:Arial">
Another example would be to extract reference data for options. For example we can extract reference data for options on FGBL (e.g. OGBL).    
</p>

In [59]:
indx_ogbl = np.where(lst_ms=='OGBL')[0][0]
print('Market Segment for OGBL :: '+str(res['MarketSegments'][indx_ogbl]['MarketSegmentID']))

Market Segment for OGBL :: 1373


<p style="color:#333333;font-family:Arial">
Once the market segment has been established we need to extract the instrument ID. For example.         
</p>

In [60]:
url = 'https://a7.deutsche-boerse.com/api/v1/rdi/XEUR/{}/{}?mode=detailed'.format(reference_date,688)
r = requests.get(url = url,  headers={'Authorization': API_TOKEN}, proxies = proxies)
res_i = r.json()

In [61]:
for x in res_i['Securities']:
    print(x['SecurityDesc'],x['SecurityID'])

FGBL.S.MAR20.JUN20 72060548975427636
FGBL.S.MAR20.SEP20 72060548975427637
FGBL.S.JUN20.SEP20 72060548975427638
FGBL SI 20200306 PS 4128839
FGBL SI 20200608 PS 4381195
FGBL SI 20200908 PS 4611674


<p style="color:#333333;font-family:Arial">
From the list above the correct contract ID needs to be selected. In this case we are interested in on the run FGBL  contracts. Which is FGBL SI 20200908 and has the instrument ID is 4611674.         
</p>